In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from pathlib import Path
from sklearn.preprocessing import LabelEncoder

In [ ]:
image_link = list(Path(r'/kaggle/input/rice-image-dataset').glob(r'**/*.jpg'))
image_name = [x.parents[0].stem for x in image_link]
image_label = LabelEncoder().fit_transform(image_name)

In [ ]:
df = pd.DataFrame()
df['link'] = np.array(image_link)
df['name'] = image_name
df['label'] = image_label

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['name'].unique()

In [ ]:
df.name.value_counts().plot(kind='bar', figsize=(12, 8))

In [4]:
import cv2

In [ ]:
num_images = 10
sample_df = df.sample(num_images)
for index, row in sample_df.iterrows():
    image = cv2.imread(str(row['link']))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.figure()
    plt.imshow(image)
    plt.title(row['name'])
    plt.axis('off')

plt.show()

In [ ]:
def get_image_size(row):
    image = cv2.imread(str(row['link']))
    height, width, _ = image.shape
    return (width, height)

In [ ]:
image_size = df.apply(get_image_size, axis=1)

In [ ]:
image_size.unique()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [ ]:
target_size = (100, 100)

In [ ]:
df['link'] = df['link'].astype(str)
df['label'] = df['label'].astype(str)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
parent_directory = '/kaggle/input/rice-image-dataset/Rice_Image_Dataset/'

In [ ]:
train_data = data_gen.flow_from_dataframe(
    train_df,
    directory=parent_directory,
    x_col='link',
    y_col='label',
    target_size=target_size,
    batch_size=32,
    class_mode='categorical'
)

test_data = data_gen.flow_from_dataframe(
    test_df,
    directory=parent_directory,
    x_col='link',
    y_col='label',
    target_size=target_size,
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001, restore_best_weights=True)

In [ ]:
history = model.fit(train_data, validation_data=test_data, epochs=50, callbacks=[early_stop])

In [ ]:
history_df = pd.DataFrame(history.history)

In [ ]:
for i in range(1, 50, 20):
    history_df.loc[i:, ['loss', 'val_loss']].plot()
    history_df.loc[:, ['accuracy', 'val_accuracy']].plot()

print('Minimum validation loss: {}'.format(history_df['val_loss'].min()))
print('Maximun accuracy: {}'.format(history_df['val_accuracy'].min()))